In [1]:
%reload_ext autoreload
%autoreload 2

# Visualizing individual trajectories

When studying sequences of events (e.g care trajectories, drug sequences, ...), it might be useful to visualize individual sequences. To that end, we provide the `viz_trajectories` function to plot individual sequences given an events dataframe.

In [4]:
from eds_scikit.plot.viz_trajectories import plot_trajectories
from eds_scikit.datasets.synthetic.event_sequences import load_event_sequences

## Create synthetic dataset

In [5]:
df_events = load_event_sequences()

The `df_events` dataset contains occurrences of 12 events, derived from 7 events' families ("A", "B", "C", "D", "E", "F", "G).  
Events can be both one-time and continuous.  
An `index_date` is also provided and refers to the inclusion date of each patient in the cohort.

In [6]:
df_events.head()

,person_id,event_family,event,event_start_datetime,event_end_datetime,index_date
0,1,A,a1,2020-01-01,2020-01-02,2020-01-01
1,1,A,a2,2020-01-03,2020-01-04,2020-01-01
2,1,B,b1,2020-01-03,2020-01-06,2020-01-01
3,1,C,c1,2020-01-05,NaT,2020-01-01
4,1,C,c2,2020-01-06,2020-01-08,2020-01-01


## Plot individual sequences

Sequences can be plotted as-is, by providing the events dataframe.

In [7]:
plot_trajectories(
    df_events
)

alt.VConcatChart(...)

Further configuration can be provided, including :
- dim_mapping : dictionary to set colors and labels for each event type.
- family_col: column name of events' families.
- list_person_ids: List of specific `person_id`
- same_x_axis_scale: boolean to set all individual charts to the same scale

In [8]:
dim_mapping = {
    "a1":{"color":(255, 200, 150), "label":"eventA1"},
    "a2":{"color":(235, 200, 150), "label":"eventA2"},
    "a3":{"color":(215, 200, 150), "label":"eventA3"},
    "b1":{"color":(200, 200, 150), "label":"eventB1"},
    "c1": {"color":(50, 255, 255), "label":"eventC1"},
    "c2": {"color":(50, 200, 255), "label":"eventC2"},
    "c3": {"color":(50, 255, 200), "label":"eventC3"},
    "d1": {"color":(180, 180, 0), "label":"eventD1"},
    "d2": {"color":(180, 100, 0), "label":"eventD2"},
    "e1": {"color":(130, 60, 10), "label":"eventE1"},
    "f1": {"color":(255, 0, 0), "label":"eventF1"},
    "g1": {"color":(100, 0, 200), "label":"eventG1"},
}

In [9]:
plot_trajectories(
    df_events,
    family_col = 'event_family',
    dim_mapping = dim_mapping,
    same_x_axis_scale=True,
    title="Event sequences",
)

alt.VConcatChart(...)